In [1]:
from data_provider import data_factory,data_loader
import numpy as np

In [6]:
import pandas as pd
import numpy as np

# Read the CSV file
df = pd.read_csv('/root/code/FinFusionNet/dataset/btc/btc_cv.csv')  # Replace 'your_data.csv' with your actual file name

# Extract the target column
target = df['range5']

# Calculate the variance of the target data
variance = np.var(target)

# Print the variance
print(f"Variance of the target: {variance}")


Variance of the target: 6.87907789471551e-06


In [1]:
from torch_geometric.nn import GCNConv


In [ ]:
np.corrcoef

In [7]:
a = [[1,2,3,4],
     [4,5,6,7]]
np.corrcoef(a)

array([[1., 1.],
       [1., 1.]])

In [11]:
a = np.array(a).T
a.shape

(4, 2)

In [2]:
import argparse
args = argparse.Namespace()
args.augmentation_ratio = 0    

In [10]:
testset = data_loader.m4Dataset_btc(args,scale=False,
        data_path='btc_t_v_withf4.csv',flag="train",size=[128,0,1])


In [9]:
testset[10][0].shape

(96, 138)

In [10]:
testset[0][1].shape

(96, 1)

In [12]:
import re
import matplotlib.pyplot as plt
import numpy as np

def extract_data_from_text(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()

    blocks = []
    current_block = []
    for line in lines:
        if "Args in experiment:" in line and current_block:
            blocks.append(current_block)
            current_block = []
        current_block.append(line)
    if current_block:
        blocks.append(current_block)

    block_names = []
    avg_val_ic_values = []
    avg_test_ic_values = []
    std_val_ic_values = []
    std_test_ic_values = []

    for block in blocks:
        block_text = "".join(block)
        model_match = re.search(r"Model:\s*(\S+)", block_text)

        val_ic_matches = re.findall(r"val ic:\s*([0-9.-]+)", block_text)
        test_ic_matches = re.findall(r"test ic:\s*([0-9.-]+)", block_text)

        if model_match and val_ic_matches and test_ic_matches:
            block_names.append(model_match.group(1))
            val_ic_floats = [float(value) for value in val_ic_matches]
            test_ic_floats = [float(value) for value in test_ic_matches]

            avg_val_ic_values.append(np.mean(val_ic_floats))
            avg_test_ic_values.append(np.mean(test_ic_floats))
            std_val_ic_values.append(np.std(val_ic_floats))
            std_test_ic_values.append(np.std(test_ic_floats))

    return block_names, avg_val_ic_values, avg_test_ic_values, std_val_ic_values, std_test_ic_values

def plot_ic_values(block_names, avg_val_ic_values, avg_test_ic_values, std_val_ic_values, std_test_ic_values):
    for i, name in enumerate(block_names):
        print(f'Block: {name}')
        print(f'Average val ic: {avg_val_ic_values[i]}, Std val ic: {std_val_ic_values[i]}')
        print(f'Average test ic: {avg_test_ic_values[i]}, Std test ic: {std_test_ic_values[i]}')
        print('-' * 50)

    plt.figure(figsize=(10, 6))

    for i, name in enumerate(block_names):
        plt.scatter(i, avg_val_ic_values[i], color='blue', label='val ic' if i == 0 else "")
        plt.scatter(i, avg_test_ic_values[i], color='red', label='test ic' if i == 0 else "")
        plt.text(i, avg_val_ic_values[i], f'{name}_val', fontsize=9, ha='right')
        plt.text(i, avg_test_ic_values[i], f'{name}_test', fontsize=9, ha='right')

    plt.xlabel("Block Index")
    plt.ylabel("IC Value")
    plt.title("Average val ic and test ic values for each block")
    plt.legend()
    plt.show()
file_path = 'log_overall_arch.txt'  # Replace with the actual file path

# block_names, avg_val_ic_values, avg_test_ic_values, std_val_ic_values, std_test_ic_values = extract_data_from_text(file_path)
# plot_ic_values(block_names, avg_val_ic_values, avg_test_ic_values, std_val_ic_values, std_test_ic_values)


In [1]:
from models.MLPs import Baseline_5,Baseline_10,Baseline_15,Baseline_20,Baseline_25
import torch
from collections import OrderedDict
import numpy as np
import torch
import torch.nn as nn

In [ ]:
nn.LSTM

In [ ]:

self.encoder_layer = nn.TransformerEncoderLayer(d_model=input_size, nhead=3, dim_feedforward=512)
self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=1)


In [22]:
def custom_summary(model, *inputs):
    def forward_hook(module, input, output):
        class_name = str(module.__class__).split(".")[-1].split("'")[0]
        module_idx = len(summary)

        m_key = "%s-%i" % (class_name, module_idx + 1)
        summary[m_key] = OrderedDict()
        summary[m_key]["input_shape"] = list(input[0].size())
        summary[m_key]["output_shape"] = list(output.size())
        
        params = 0
        if hasattr(module, "weight") and hasattr(module.weight, "size"):
            params += torch.prod(torch.LongTensor(list(module.weight.size())))
            summary[m_key]["trainable"] = module.weight.requires_grad
        if hasattr(module, "bias") and hasattr(module.bias, "size"):
            params += torch.prod(torch.LongTensor(list(module.bias.size())))
        summary[m_key]["nb_params"] = params

    def register_hook(module):
        if (
            not isinstance(module, nn.Sequential)
            and not isinstance(module, nn.ModuleList)
            and not (module == model)
        ):
            hooks.append(module.register_forward_hook(forward_hook))

    device = next(model.parameters()).device
    dtype = next(model.parameters()).dtype

    batch_size = inputs[0].size(0)
    summary = OrderedDict()
    hooks = []

    model.apply(register_hook)
    model(*inputs)

    for h in hooks:
        h.remove()

    print("----------------------------------------------------------------")
    print("Layer (type)               Output Shape         Param #")
    print("================================================================")
    total_params = 0
    for layer in summary:
        line_new = "{:30}  {:25} {:15}".format(
            layer,
            str(summary[layer]["output_shape"]),
            "{0:,}".format(summary[layer]["nb_params"]),
        )
        total_params += summary[layer]["nb_params"]
        print(line_new)

    print("================================================================")
    print("Total params: {0:,}".format(total_params))
    print("----------------------------------------------------------------")


In [23]:
input1 = torch.randn(1024,1, 138)
model = Baseline_5.Model()

custom_summary(model, input1, 0,0,0)


----------------------------------------------------------------
Layer (type)               Output Shape         Param #
Linear-1                        [1024, 1, 64]             8,896          
ReLU-2                          [1024, 1, 64]             0              
Linear-3                        [1024, 1, 32]             2,080          
ReLU-4                          [1024, 1, 32]             0              
Linear-5                        [1024, 1, 16]             528            
ReLU-6                          [1024, 1, 16]             0              
Linear-7                        [1024, 1, 8]              136            
ReLU-8                          [1024, 1, 8]              0              
Linear-9                        [1024, 1, 4]              36             
ReLU-10                         [1024, 1, 4]              0              
Linear-11                       [1024, 1, 2]              10             
ReLU-12                         [1024, 1, 2]              0      

In [ ]:
self.pool1d = nn.MaxPool1d(kernel_size=2)


In [24]:
input1 = torch.randn(2,1, 138)
model = Baseline_25.Model()

custom_summary(model, input1, 0,0,0)


----------------------------------------------------------------
Layer (type)               Output Shape         Param #
Linear-1                        [2, 1, 256]               35,584         
BatchNorm1d-2                   [2, 256, 1]               512            
ReLU-3                          [2, 1, 256]               0              
Linear-4                        [2, 1, 518]               133,126        
BatchNorm1d-5                   [2, 518, 1]               1,036          
ReLU-6                          [2, 1, 518]               0              
Linear-7                        [2, 1, 1024]              531,456        
BatchNorm1d-8                   [2, 1024, 1]              2,048          
ReLU-9                          [2, 1, 1024]              0              
Linear-10                       [2, 1, 2048]              2,099,200      
BatchNorm1d-11                  [2, 2048, 1]              4,096          
ReLU-12                         [2, 1, 2048]              0      

In [2]:
import torch

def sliding_window_slice(tensor, window_size, stride=1):
    """
    Performs sliding window slicing on a PyTorch tensor.
    
    Args:
        tensor (torch.Tensor): The input tensor of shape (n, x, y).
        window_size (int): The size of the sliding window.
        stride (int): The stride of the sliding window.

    Returns:
        List[torch.Tensor]: List of sliced tensors.
    """
    n, x, y = tensor.shape
    sliced_tensors = []

    for i in range(0, x - window_size + 1, stride):
        for j in range(0, y - window_size + 1, stride):
            slice_ = tensor[:, i:i + window_size, j:j + window_size]
            sliced_tensors.append(slice_)

    return sliced_tensors

# Example usage:
tensor = torch.randn(5, 10, 10)  # Example tensor with shape (n, x, y)
window_size = 5
stride = 1
sliced_tensors = sliding_window_slice(tensor, window_size, stride)

# Print the shapes of the sliced tensors
for idx, st in enumerate(sliced_tensors):
    print(f"Sliced tensor {idx} shape: {st.shape}")


Sliced tensor 0 shape: torch.Size([5, 5, 5])
Sliced tensor 1 shape: torch.Size([5, 5, 5])
Sliced tensor 2 shape: torch.Size([5, 5, 5])
Sliced tensor 3 shape: torch.Size([5, 5, 5])
Sliced tensor 4 shape: torch.Size([5, 5, 5])
Sliced tensor 5 shape: torch.Size([5, 5, 5])
Sliced tensor 6 shape: torch.Size([5, 5, 5])
Sliced tensor 7 shape: torch.Size([5, 5, 5])
Sliced tensor 8 shape: torch.Size([5, 5, 5])
Sliced tensor 9 shape: torch.Size([5, 5, 5])
Sliced tensor 10 shape: torch.Size([5, 5, 5])
Sliced tensor 11 shape: torch.Size([5, 5, 5])
Sliced tensor 12 shape: torch.Size([5, 5, 5])
Sliced tensor 13 shape: torch.Size([5, 5, 5])
Sliced tensor 14 shape: torch.Size([5, 5, 5])
Sliced tensor 15 shape: torch.Size([5, 5, 5])
Sliced tensor 16 shape: torch.Size([5, 5, 5])
Sliced tensor 17 shape: torch.Size([5, 5, 5])
Sliced tensor 18 shape: torch.Size([5, 5, 5])
Sliced tensor 19 shape: torch.Size([5, 5, 5])
Sliced tensor 20 shape: torch.Size([5, 5, 5])
Sliced tensor 21 shape: torch.Size([5, 5, 5]

In [ ]:
class TransformerRegressor(nn.Module):
    def __init__(self, input_dim, output_dim, nhead, num_encoder_layers, dim_feedforward):
        super(TransformerRegressor, self).__init__()
        self.encoder_layer = nn.TransformerEncoderLayer(d_model=input_dim, nhead=nhead, dim_feedforward=dim_feedforward)
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=num_encoder_layers)
        self.regressor = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        x = self.transformer_encoder(x)
        x = self.regressor(x[-1])  # use the last time step
        return x

In [24]:
import pandas as pd
import numpy as np


# Load the CSV file
csv_file = '/root/code/FinFusionNet/dataset/btc/btc_t_v_withf4ct.csv'  # Replace with your CSV file path
data = pd.read_csv(csv_file)

In [25]:
data.shape

(500000, 141)

In [2]:
range5 = data['range5']


In [20]:
np.mean(range5)

-7.510849319229187e-06

In [23]:
mean = 0  # Replace with your desired mean
std_dev = 0.002  # Replace with your desired standard deviation
normal_array = np.random.normal(mean, std_dev, len(range5))

# Calculate the Pearson correlation coefficient
correlation_coefficient, _ = np.corrcoef(range5, normal_array)

# Print the result
print(f'Pearson correlation coefficient: {correlation_coefficient}')


Pearson correlation coefficient: [1.00000000e+00 4.09327437e-04]


In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class MaxAbsolutePooling1D(nn.Module):
    def __init__(self, kernel_size, stride=None, padding=0, dilation=1):
        super(MaxAbsolutePooling1D, self).__init__()
        self.kernel_size = kernel_size
        self.stride = stride if stride is not None else kernel_size
        self.padding = padding
        self.dilation = dilation

    def forward(self, x):
        # Compute the absolute values
        abs_x = torch.abs(x)
        
        # Find the indices of the maximum absolute values
        pooled_abs, indices = F.max_pool1d(abs_x, self.kernel_size, self.stride, 
                                           self.padding, self.dilation, 
                                           return_indices=True)
        
        # Create a tensor of indices for gathering
        batch_size, channels, output_length = pooled_abs.shape
        batch_indices = torch.arange(batch_size).view(-1, 1, 1).expand(-1, channels, output_length)
        channel_indices = torch.arange(channels).view(1, -1, 1).expand(batch_size, -1, output_length)
        
        # Gather the original values using the indices
        output = x[batch_indices, channel_indices, indices]
        
        return output

# Example usage
pool = MaxAbsolutePooling1D(kernel_size=2, stride=2)
input_tensor = torch.randn(2, 3, 3)  # Batch size 2, 3 channels, sequence length 8
output = pool(input_tensor)

print("Input shape:", input_tensor.shape)
print("Input:")
print(input_tensor)
print("\nOutput shape:", output.shape)
print("Output:")
print(output)
raw = F.max_pool1d(input_tensor, 2, 2)
print(f"\nraw {raw}")

Input shape: torch.Size([2, 3, 3])
Input:
tensor([[[ 0.3511, -1.4718, -0.5558],
         [ 1.3155,  1.3483, -1.0512],
         [-1.5278,  1.0408, -0.1548]],

        [[-0.9161, -0.8894,  1.5297],
         [-0.2839,  0.9070, -0.4745],
         [-0.7224,  0.5794, -0.2973]]])

Output shape: torch.Size([2, 3, 1])
Output:
tensor([[[-1.4718],
         [ 1.3483],
         [-1.5278]],

        [[-0.9161],
         [ 0.9070],
         [-0.7224]]])

raw tensor([[[ 0.3511],
         [ 1.3483],
         [ 1.0408]],

        [[-0.8894],
         [ 0.9070],
         [ 0.5794]]])


In [1]:
import tensorflow as tf

# Define a simple computation
a = tf.constant(3.0)
b = tf.constant(4.0)
c = a * b

# Print the value of c
tf.print("The value of c is:", c)

# Continue with another operation
d = c + 2.0

# Print the value of d
tf.print("The value of d is:", d)


2024-08-12 14:25:43.627312: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-12 14:25:44.521217: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


The value of c is: 12
The value of d is: 14


2024-08-12 14:25:47.078527: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 20454 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:81:00.0, compute capability: 8.6
